# Quantum Dynamics Lab
## Visualizing State Evolution, Heisenberg Picture, and Ehrenfest Theorem

This notebook explores the dynamical aspects of quantum mechanics:

| Module | Topic | Key Concepts |
|--------|-------|-------------|
| **A** | Quantum Condition | $[\hat{x}, \hat{p}] = i\hbar$, commutators |
| **B** | Schrödinger Picture | State evolution $\|\psi(t)\rangle = e^{-iHt/\hbar}\|\psi(0)\rangle$ |
| **C** | Heisenberg & Ehrenfest | Operator evolution, classical limit |
| **D** | Interaction Picture | $H = H_0 + V$ splitting |

---

In [ ]:
# ============================================================
# IMPORTS
# ============================================================
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.linalg import expm
from IPython.display import HTML

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 11

# Physical constants (using ℏ = 1 unless otherwise noted)
HBAR = 1.0

---
# Module A: Quantum Condition and Commutators

## Dirac's Quantum Condition

The fundamental commutation relation between position and momentum:

$$[\hat{x}, \hat{p}] = \hat{x}\hat{p} - \hat{p}\hat{x} = i\hbar$$

### Physical Meaning
- Position and momentum cannot be simultaneously measured with arbitrary precision
- The order of operations matters: $\hat{x}\hat{p} \neq \hat{p}\hat{x}$
- This is the foundation of Heisenberg's uncertainty principle

### Finite-Dimensional Approximation

In a continuous space, $[x, p] = i\hbar$ exactly. But in a **finite-dimensional** (truncated) space, we can only approximate this. We'll use:

- **Position basis**: $X_{nm} = x_n \delta_{nm}$ (diagonal)
- **Momentum via finite differences**: $P = -i\hbar \frac{d}{dx} \approx$ tridiagonal matrix

In [ ]:
def build_position_operator(x):
    """
    Build position operator X in position basis.
    
    In position basis, X is diagonal with eigenvalues x_i.
    X|x_i⟩ = x_i|x_i⟩
    
    Parameters:
    -----------
    x : ndarray
        Position grid points
        
    Returns:
    --------
    X : ndarray
        Diagonal position operator matrix
    """
    return np.diag(x)


def build_momentum_operator(N, dx, hbar=HBAR):
    """
    Build momentum operator P using finite differences.
    
    P = -iℏ d/dx is approximated using central differences:
    (dψ/dx)_i ≈ (ψ_{i+1} - ψ_{i-1}) / (2Δx)
    
    This gives a tridiagonal matrix with:
    - Main diagonal: 0
    - Super-diagonal: +iℏ/(2Δx)
    - Sub-diagonal: -iℏ/(2Δx)
    
    Parameters:
    -----------
    N : int
        Number of grid points
    dx : float
        Grid spacing
    hbar : float
        Reduced Planck's constant
        
    Returns:
    --------
    P : ndarray (complex)
        Momentum operator matrix
    """
    # Coefficient from -iℏ × 1/(2Δx)
    coeff = -1j * hbar / (2 * dx)
    
    # Build tridiagonal: +1 on super-diagonal, -1 on sub-diagonal
    P = coeff * (np.diag(np.ones(N-1), k=1) - np.diag(np.ones(N-1), k=-1))
    
    return P


def commutator(A, B):
    """
    Compute the commutator [A, B] = AB - BA.
    
    Properties:
    - [A, B] = -[B, A] (antisymmetric)
    - [A, A] = 0
    - For Hermitian A, B: [A, B] is anti-Hermitian
    """
    return A @ B - B @ A

In [ ]:
# ============================================================
# CONSTRUCT X, P OPERATORS
# ============================================================

# Grid parameters
N = 50  # Number of grid points
x_max = 5.0
x = np.linspace(-x_max, x_max, N)
dx = x[1] - x[0]

# Build operators
X = build_position_operator(x)
P = build_momentum_operator(N, dx)

print(f"Grid: {N} points, Δx = {dx:.4f}")
print(f"X shape: {X.shape}")
print(f"P shape: {P.shape}")

In [ ]:
# ============================================================
# COMPUTE COMMUTATOR [X, P]
# ============================================================

XP_comm = commutator(X, P)

# Theoretical result: [X, P] = iℏI
theoretical = 1j * HBAR * np.eye(N)

# Compare
print("Commutator [X, P] Analysis:")
print("="*50)
print(f"Expected: iℏ·I = i × {HBAR} × I")
print(f"\nDiagonal elements of [X,P] (first 5):")
print(np.diag(XP_comm)[:5])
print(f"\nExpected diagonal: {1j * HBAR}")

In [ ]:
# ============================================================
# VISUALIZATION: Heatmaps of X, P, [X,P]
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Position operator (real, diagonal)
ax1 = axes[0]
im1 = ax1.imshow(np.real(X), cmap='RdBu_r', aspect='equal')
ax1.set_title('Position Operator X (Real)')
ax1.set_xlabel('Column')
ax1.set_ylabel('Row')
plt.colorbar(im1, ax=ax1)

# Momentum operator (imaginary part shown)
ax2 = axes[1]
im2 = ax2.imshow(np.imag(P), cmap='RdBu_r', aspect='equal')
ax2.set_title('Momentum Operator P (Imaginary)')
ax2.set_xlabel('Column')
ax2.set_ylabel('Row')
plt.colorbar(im2, ax=ax2)

# Commutator (imaginary part should be ℏ on diagonal)
ax3 = axes[2]
im3 = ax3.imshow(np.imag(XP_comm), cmap='viridis', aspect='equal')
ax3.set_title('[X, P] (Imaginary Part)')
ax3.set_xlabel('Column')
ax3.set_ylabel('Row')
plt.colorbar(im3, ax=ax3, label='Should be ℏ = 1 on diagonal')

plt.tight_layout()
plt.savefig('commutator_heatmaps.png', dpi=150)
plt.show()

In [ ]:
# ============================================================
# TRUNCATION ERROR ANALYSIS
# ============================================================

# Compute difference from ideal
error = XP_comm - theoretical

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Heatmap of error
im = ax1.imshow(np.abs(error), cmap='hot')
ax1.set_title('[X, P] - iℏI (Absolute Error)')
ax1.set_xlabel('Column')
ax1.set_ylabel('Row')
plt.colorbar(im, ax=ax1)

# Diagonal elements vs expected
diag_actual = np.imag(np.diag(XP_comm))
diag_expected = HBAR * np.ones(N)

ax2.plot(range(N), diag_actual, 'b-', linewidth=2, label='Actual Im([X,P]ᵢᵢ)')
ax2.axhline(HBAR, color='r', linestyle='--', label=f'Expected: ℏ = {HBAR}')
ax2.set_xlabel('Diagonal index i')
ax2.set_ylabel('Value')
ax2.set_title('Diagonal Elements of [X, P]')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('truncation_error.png', dpi=150)
plt.show()

print(f"\nMax error in diagonal: {np.max(np.abs(diag_actual - HBAR)):.6f}")
print(f"Errors are larger at boundaries due to finite grid effects.")

---
# Module B: State Evolution in Schrödinger Picture

## Time Evolution

The Schrödinger equation:
$$i\hbar\frac{d|\psi\rangle}{dt} = \hat{H}|\psi\rangle$$

Has formal solution:
$$|\psi(t)\rangle = e^{-i\hat{H}t/\hbar}|\psi(0)\rangle = \hat{U}(t)|\psi(0)\rangle$$

### Methods to Compute $e^{-iHt}$

1. **Eigendecomposition**: If $H|E_n\rangle = E_n|E_n\rangle$:
   $$e^{-iHt} = \sum_n e^{-iE_nt/\hbar}|E_n\rangle\langle E_n|$$

2. **Matrix exponential**: Use `scipy.linalg.expm(-1j*H*t)`

In [ ]:
# ============================================================
# HELPER FUNCTIONS
# ============================================================

def normalize(psi):
    """Normalize a quantum state."""
    return psi / np.sqrt(np.vdot(psi, psi))

def expectation_value(psi, A):
    """Compute ⟨ψ|A|ψ⟩."""
    return np.real(np.vdot(psi, A @ psi))

def evolve_state(psi0, H, t, hbar=HBAR):
    """
    Evolve state using Schrödinger equation.
    
    |ψ(t)⟩ = exp(-iHt/ℏ)|ψ(0)⟩
    
    Parameters:
    -----------
    psi0 : ndarray
        Initial state
    H : ndarray
        Hamiltonian matrix
    t : float
        Time
    hbar : float
        Planck's constant
        
    Returns:
    --------
    psi_t : ndarray
        State at time t
    """
    U = expm(-1j * H * t / hbar)  # Time evolution operator
    return U @ psi0

## System 1: Two-Level System (Qubit)

### Hamiltonian
$$H = \frac{\omega_0}{2}\sigma_z + \frac{\Omega}{2}\sigma_x = \begin{pmatrix} \omega_0/2 & \Omega/2 \\ \Omega/2 & -\omega_0/2 \end{pmatrix}$$

- $\omega_0$: energy splitting
- $\Omega$: Rabi frequency (coupling strength)

### Physics
Starting in $|0\rangle$ (ground state), the system oscillates between $|0\rangle$ and $|1\rangle$ with frequency related to $\Omega$.

In [ ]:
# Pauli matrices
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
I2 = np.eye(2, dtype=complex)

def two_level_hamiltonian(omega_0, Omega):
    """
    Create 2-level system Hamiltonian.
    
    H = (ω₀/2)σz + (Ω/2)σx
    
    Physical interpretation:
    - ω₀: level splitting (bare frequency)
    - Ω: coupling strength (Rabi frequency)
    """
    return (omega_0/2) * sigma_z + (Omega/2) * sigma_x

In [ ]:
# ============================================================
# RABI OSCILLATIONS
# ============================================================

omega_0 = 0.0   # Zero detuning (resonance)
Omega = 1.0     # Rabi frequency

H_2level = two_level_hamiltonian(omega_0, Omega)

# Initial state: ground state |0⟩
psi0 = np.array([1, 0], dtype=complex)

# Excited state projector for P_excited
excited = np.array([0, 1], dtype=complex)
P_excited_op = np.outer(excited, excited)  # |1⟩⟨1|

# Time evolution
times = np.linspace(0, 4*np.pi, 200)
P_excited = []

for t in times:
    psi_t = evolve_state(psi0, H_2level, t)
    P_excited.append(expectation_value(psi_t, P_excited_op))

P_excited = np.array(P_excited)

In [ ]:
# Plot Rabi oscillations
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(times, P_excited, 'b-', linewidth=2, label='P(excited)')
ax.plot(times, 1 - P_excited, 'r--', linewidth=2, label='P(ground)')

ax.set_xlabel('Time')
ax.set_ylabel('Probability')
ax.set_title(f'Rabi Oscillations (ω₀={omega_0}, Ω={Omega})')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_ylim(-0.05, 1.05)

# Mark Rabi period
T_rabi = 2*np.pi / np.sqrt(omega_0**2 + Omega**2)
ax.axvline(T_rabi, color='green', linestyle=':', label=f'T_Rabi = {T_rabi:.2f}')

plt.tight_layout()
plt.savefig('rabi_oscillations.png', dpi=150)
plt.show()

print(f"Rabi period T = 2π/Ω_eff = {T_rabi:.4f}")

## System 2: 1D Harmonic Oscillator

### Hamiltonian (using ladder operators)
$$H = \hbar\omega\left(a^\dagger a + \frac{1}{2}\right)$$

We'll use a coherent state (displaced ground state) as initial condition.

In [ ]:
def build_ladder_operators(N):
    """
    Build creation and annihilation operators.
    
    a|n⟩ = √n|n-1⟩
    a†|n⟩ = √(n+1)|n+1⟩
    """
    a = np.diag(np.sqrt(np.arange(1, N)), k=1).astype(complex)
    a_dag = np.conj(a.T)
    return a, a_dag

def harmonic_hamiltonian(N, omega=1.0):
    """
    Build harmonic oscillator Hamiltonian in Fock basis.
    
    H = ℏω(a†a + 1/2)
    """
    a, a_dag = build_ladder_operators(N)
    number_op = a_dag @ a
    return HBAR * omega * (number_op + 0.5 * np.eye(N))

In [ ]:
# Build oscillator
N_osc = 30  # Truncation
omega = 1.0

H_osc = harmonic_hamiltonian(N_osc, omega)
a, a_dag = build_ladder_operators(N_osc)

# Position and momentum operators
X_osc = (a + a_dag) / np.sqrt(2)
P_osc = 1j * (a_dag - a) / np.sqrt(2)

# Initial state: coherent state (displaced vacuum)
# |α⟩ = exp(-|α|²/2) Σ (αⁿ/√n!)|n⟩
alpha = 2.0  # Displacement

def coherent_state(alpha, N):
    """Create coherent state |α⟩ in Fock basis."""
    from math import factorial
    psi = np.zeros(N, dtype=complex)
    for n in range(N):
        psi[n] = (alpha**n / np.sqrt(factorial(n)))
    psi *= np.exp(-np.abs(alpha)**2 / 2)
    return normalize(psi)

psi0_osc = coherent_state(alpha, N_osc)

print(f"Coherent state |α={alpha}⟩ created")
print(f"⟨X⟩₀ = {expectation_value(psi0_osc, X_osc):.4f} (expected: √2·Re(α) = {np.sqrt(2)*alpha:.4f})")

In [ ]:
# ============================================================
# ANIMATION: Probability distribution in Fock space
# ============================================================

T_period = 2 * np.pi / omega
t_anim = np.linspace(0, 2*T_period, 100)

fig, ax = plt.subplots(figsize=(10, 5))

def init():
    ax.set_xlim(-0.5, N_osc-0.5)
    ax.set_ylim(0, 0.4)
    return []

def animate(frame):
    ax.clear()
    
    t = t_anim[frame]
    psi_t = evolve_state(psi0_osc, H_osc, t)
    probs = np.abs(psi_t)**2
    
    ax.bar(range(N_osc), probs, color='steelblue', alpha=0.8)
    ax.set_xlabel('Fock state |n⟩')
    ax.set_ylabel('Probability')
    ax.set_title(f'Coherent State Evolution (t = {t:.2f}, T = {T_period:.2f})')
    ax.set_xlim(-0.5, 15.5)
    ax.set_ylim(0, 0.35)
    ax.grid(True, alpha=0.3)
    
    return []

anim = FuncAnimation(fig, animate, init_func=init, frames=len(t_anim), interval=50, blit=False)
plt.close()
HTML(anim.to_jshtml())

---
# Module C: Heisenberg Picture and Ehrenfest Theorem

## Heisenberg Picture

Instead of evolving states, we evolve **operators**:

$$\hat{A}_H(t) = e^{i\hat{H}t/\hbar}\hat{A}e^{-i\hat{H}t/\hbar}$$

The **Heisenberg equation of motion**:
$$\frac{d\hat{A}_H}{dt} = \frac{i}{\hbar}[\hat{H}, \hat{A}_H] + \left(\frac{\partial\hat{A}}{\partial t}\right)_H$$

## Ehrenfest Theorem

For position and momentum in a potential $V(x)$:

$$\frac{d\langle x\rangle}{dt} = \frac{\langle p\rangle}{m}$$
$$\frac{d\langle p\rangle}{dt} = -\langle\nabla V\rangle$$

These look like **classical equations of motion**! For well-localized wave packets, quantum mechanics approaches classical mechanics.

In [ ]:
def heisenberg_evolve(A, H, t, hbar=HBAR):
    """
    Evolve operator in Heisenberg picture.
    
    A_H(t) = U†(t) A U(t) = exp(iHt/ℏ) A exp(-iHt/ℏ)
    
    Parameters:
    -----------
    A : ndarray
        Operator at t=0
    H : ndarray
        Hamiltonian
    t : float
        Time
        
    Returns:
    --------
    A_H : ndarray
        Heisenberg-picture operator at time t
    """
    U = expm(-1j * H * t / hbar)
    U_dag = np.conj(U.T)
    return U_dag @ A @ U

In [ ]:
# ============================================================
# VERIFY: Both pictures give same expectation values
# ============================================================

t_test = 1.5

# Schrödinger picture: evolve state
psi_t_schrod = evolve_state(psi0_osc, H_osc, t_test)
exp_x_schrod = expectation_value(psi_t_schrod, X_osc)
exp_p_schrod = expectation_value(psi_t_schrod, P_osc)

# Heisenberg picture: evolve operators
X_H = heisenberg_evolve(X_osc, H_osc, t_test)
P_H = heisenberg_evolve(P_osc, H_osc, t_test)
exp_x_heisen = expectation_value(psi0_osc, X_H)  # Use initial state!
exp_p_heisen = expectation_value(psi0_osc, P_H)

print("Comparison of Pictures at t = 1.5:")
print("="*50)
print(f"{'Quantity':<20} {'Schrödinger':>15} {'Heisenberg':>15}")
print("-"*50)
print(f"{'⟨X⟩':<20} {exp_x_schrod:>15.6f} {exp_x_heisen:>15.6f}")
print(f"{'⟨P⟩':<20} {exp_p_schrod:>15.6f} {exp_p_heisen:>15.6f}")
print("="*50)
print(f"\n✓ Agreement confirms equivalence of pictures!")

In [ ]:
# ============================================================
# EHRENFEST THEOREM VERIFICATION
# ============================================================

times = np.linspace(0, 4*np.pi, 200)

exp_x_list = []
exp_p_list = []

for t in times:
    psi_t = evolve_state(psi0_osc, H_osc, t)
    exp_x_list.append(expectation_value(psi_t, X_osc))
    exp_p_list.append(expectation_value(psi_t, P_osc))

exp_x = np.array(exp_x_list)
exp_p = np.array(exp_p_list)

# Classical trajectory for harmonic oscillator
# x(t) = x₀ cos(ωt), p(t) = -m·ω·x₀ sin(ωt)
# For coherent state: x₀ = √2·Re(α), p₀ = √2·Im(α)
x0_classical = np.sqrt(2) * np.real(alpha)
p0_classical = np.sqrt(2) * np.imag(alpha)  # = 0 for real α

x_classical = x0_classical * np.cos(omega * times)
p_classical = -x0_classical * omega * np.sin(omega * times)

In [ ]:
# ============================================================
# PLOT: Quantum vs Classical
# ============================================================

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Position
ax1.plot(times, exp_x, 'b-', linewidth=2, label='Quantum ⟨X⟩')
ax1.plot(times, x_classical, 'r--', linewidth=2, label='Classical x(t)')
ax1.set_ylabel('Position')
ax1.set_title('Ehrenfest Theorem: Quantum Expectation Values Follow Classical Motion')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Momentum
ax2.plot(times, exp_p, 'b-', linewidth=2, label='Quantum ⟨P⟩')
ax2.plot(times, p_classical, 'r--', linewidth=2, label='Classical p(t)')
ax2.set_xlabel('Time')
ax2.set_ylabel('Momentum')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ehrenfest_theorem.png', dpi=150)
plt.show()

# Compute difference
x_diff = np.max(np.abs(exp_x - x_classical))
p_diff = np.max(np.abs(exp_p - p_classical))
print(f"Max |⟨X⟩ - x_classical|: {x_diff:.6f}")
print(f"Max |⟨P⟩ - p_classical|: {p_diff:.6f}")
print("\n✓ Coherent states maintain their shape and follow classical trajectories!")

In [ ]:
# ============================================================
# ANIMATION: Classical trajectory with quantum expectation
# ============================================================

fig, ax = plt.subplots(figsize=(8, 8))

def init():
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    return []

def animate(frame):
    ax.clear()
    
    # Draw phase space trajectory up to current time
    ax.plot(exp_x[:frame+1], exp_p[:frame+1], 'b-', linewidth=1, alpha=0.5)
    
    # Current quantum point
    ax.scatter([exp_x[frame]], [exp_p[frame]], s=100, color='blue', 
               label='Quantum ⟨X⟩, ⟨P⟩', zorder=5)
    
    # Classical trajectory
    ax.plot(x_classical[:frame+1], p_classical[:frame+1], 'r--', linewidth=1, alpha=0.5)
    ax.scatter([x_classical[frame]], [p_classical[frame]], s=100, color='red', 
               marker='x', linewidths=3, label='Classical', zorder=5)
    
    ax.set_xlabel('Position X')
    ax.set_ylabel('Momentum P')
    ax.set_title(f'Phase Space (t = {times[frame]:.2f})')
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    return []

anim = FuncAnimation(fig, animate, init_func=init, frames=range(0, len(times), 2),
                     interval=50, blit=False)
plt.close()
HTML(anim.to_jshtml())

---
# Module D: Interaction Picture (Optional)

## Motivation

When $H = H_0 + V$ where $H_0$ is "simple" (e.g., diagonal) and $V$ is a perturbation:

- **Schrödinger**: All evolution in states
- **Heisenberg**: All evolution in operators
- **Interaction**: $H_0$ evolution in operators, $V$ evolution in states

## Transformation

$$|\psi_I(t)\rangle = e^{iH_0 t/\hbar}|\psi_S(t)\rangle$$

$$\hat{A}_I(t) = e^{iH_0 t/\hbar}\hat{A}e^{-iH_0 t/\hbar}$$

In [ ]:
def interaction_picture_state(psi_S, H0, t, hbar=HBAR):
    """
    Transform Schrödinger state to interaction picture.
    
    |ψ_I(t)⟩ = exp(iH₀t/ℏ)|ψ_S(t)⟩
    
    This removes the "trivial" H₀ evolution.
    """
    U0_dag = expm(1j * H0 * t / hbar)
    return U0_dag @ psi_S

In [ ]:
# ============================================================
# 2-LEVEL SYSTEM WITH H = H₀ + V
# ============================================================

# H₀ = (ω₀/2)σz (free evolution, diagonal)
# V = (Ω/2)σx (coupling, off-diagonal)

omega_0 = 2.0  # Level splitting
Omega = 0.5    # Coupling (perturbation)

H0 = (omega_0/2) * sigma_z
V = (Omega/2) * sigma_x
H_full = H0 + V

print("H₀ (diagonal):")
print(H0)
print("\nV (perturbation):")
print(V)

In [ ]:
# Time evolution in both pictures
psi0 = np.array([1, 0], dtype=complex)  # Start in ground state
times = np.linspace(0, 8*np.pi, 300)

# Schrödinger picture populations
pop_excited_S = []
pop_ground_S = []

# Interaction picture populations
pop_excited_I = []
pop_ground_I = []

for t in times:
    # Schrödinger picture
    psi_S = evolve_state(psi0, H_full, t)
    pop_ground_S.append(np.abs(psi_S[0])**2)
    pop_excited_S.append(np.abs(psi_S[1])**2)
    
    # Interaction picture
    psi_I = interaction_picture_state(psi_S, H0, t)
    pop_ground_I.append(np.abs(psi_I[0])**2)
    pop_excited_I.append(np.abs(psi_I[1])**2)

pop_excited_S = np.array(pop_excited_S)
pop_excited_I = np.array(pop_excited_I)

In [ ]:
# ============================================================
# TWO-PANEL COMPARISON
# ============================================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Schrödinger picture
ax1.plot(times, pop_ground_S, 'b-', linewidth=2, label='P(ground)')
ax1.plot(times, pop_excited_S, 'r-', linewidth=2, label='P(excited)')
ax1.set_xlabel('Time')
ax1.set_ylabel('Population')
ax1.set_title('Schrödinger Picture')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-0.05, 1.05)

# Interaction picture
ax2.plot(times, pop_ground_I, 'b-', linewidth=2, label='P(ground)')
ax2.plot(times, pop_excited_I, 'r-', linewidth=2, label='P(excited)')
ax2.set_xlabel('Time')
ax2.set_ylabel('Population')
ax2.set_title('Interaction Picture')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-0.05, 1.05)

plt.suptitle(f'Same Physics, Different Pictures (ω₀={omega_0}, Ω={Omega})', fontsize=14)
plt.tight_layout()
plt.savefig('interaction_picture.png', dpi=150)
plt.show()

print("The POPULATIONS are identical in both pictures!")
print(f"Max difference: {np.max(np.abs(pop_excited_S - pop_excited_I)):.2e}")

---
# Summary

## Key Results

| Module | Key Finding |
|--------|-------------|
| **A** | $[X, P] \approx i\hbar I$ with boundary truncation errors |
| **B** | State evolution shows Rabi oscillations and coherent state dynamics |
| **C** | Schrödinger and Heisenberg give identical $\langle A \rangle$; Ehrenfest links to classical |
| **D** | Interaction picture gives same physics with different time dependence |

## Physical Insights

1. **Commutators** encode fundamental quantum uncertainty
2. **Time evolution** is unitary: probability is conserved
3. **Ehrenfest theorem**: Quantum → Classical for well-localized states
4. **Pictures are equivalent**: Choose based on computational convenience